Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

In [1]:
import os,sys,numpy as np,skimage,nibabel as nib, nibabel.processing, nibabel.funcs
'''
import sys
import numpy as np
import ntpath
import warnings
import tempfile
import struct
import copy
import gc
import shutil
'''
from skimage import measure, filters, morphology
from skimage.transform import rescale, resize

In [2]:
def get_cube_type(max_size):
    tum_size_map=[
        {'range':[0,5],'cube_dim':15,'label':'small'},
        {'range':[5,10],'cube_dim':20,'label':'medium'},
        {'range':[10,20],'cube_dim':30,'label':'large'},
        {'range':[20,60],'cube_dim':70,'label':'ex_large'}
    ]
    for e in tum_size_map:
        rng=e['range']
        if max_size>=rng[0] and max_size<rng[1]: return e
    return None
    
def resample_image_111(img_in,is_mask):
    order=0 if is_mask else 3
    return nib.processing.resample_to_output(img_in,[1,1,1],order=order)

def conform_image_111(img_in,img_ref,is_mask):
    order=0 if is_mask else 3
    return nibabel.processing.conform(img_in,img_ref.shape,voxel_size=(1.0,1.0,1.0),order=order)

def split_image(file_im,files_masks):
    try:
        im0=nib.load(file_im)
        if len(im0.shape)<3 or len(im0.shape)>4:
            print('input image shape is <3 or >4!'); return None      
        if len(im0.shape)==4:
            im=resample_image_111(nibabel.funcs.four_to_three(im0)[0],False)
        else:
            im=resample_image_111(im0,False)           
        masks=[]
        for mf in files_masks:
            msk=nib.load(mf)
            if msk.shape[:3] != im0.shape[:3]:
                print('ERROR: Dimensions of input file {} and mask {} do not match!'.format(im.shape,msk.shape))
                return None
            masks+=[conform_image_111(msk,im,True)]
        return im,masks
    except:
        print('ERROR: cannot read input file(s)')
        print(sys.exc_info()[1])
        return None
       
def get_subimages(img,mask):
    props=skimage.measure.regionprops(mask.get_fdata().astype('int'))
    bb=props[0].bbox
    bb_range=[(bb[i],bb[i+3]) for i in range(0,3) ]
    bb_center=[ bb_range[i][0]+(bb_range[i][1]-bb_range[i][0])/2 for i in range (0,3) ]
    
    #print('bbox',bb_range)
    dims=np.array([bb[3]-bb[0],bb[4]-bb[1],bb[5]-bb[2]]).astype('int')
    #print('bbox size:',dims)
    #print('bbox center:',bb_center)
    
    tp=get_cube_type(dims.max())
    if tp is None: return None,None
    
    cube_dim=tp['cube_dim']
    #print('cube_dim',cube_dim)
 
    rng=[]
    for i in range (3):
        d0=int(bb_center[i]-(cube_dim/2))
        d1=int(d0+cube_dim)
        #print('axis',i,'bounds',d0,d1)
        rng+=[[d0,d1]]
 
    print('output region:', rng)
    return nifti_subimage_111(img,rng,False),nifti_subimage_111(mask,rng,True)

#bounds are a 2d array [[st,en],[st,en],[st,en]]
def nifti_subimage_111(img,bounds,is_mask):
    voxels=img.get_fdata()
    ish=np.array(voxels.shape)
    dims=np.array([bounds[i][1]-bounds[i][0] for i in range(3)])
    
    subim=np.zeros(dims)
    ist,ien=[0,0,0],[0,0,0]
    sst,sen=[0,0,0],[0,0,0]
    
    for i in range(3):
        i0,i1=0,ish[i]
        s0,s1=bounds[i][0],bounds[i][1]
        if s1<i0 or s0>i1: print('exiting'); return None #the sub-range is outside original image
        ist[i]=s0 if s0>=i0 else 0
        ien[i]=s1 if s1<=i1 else i1
        sst[i]=0 if s0>=i0 else i0-s0
        sen[i]=dims[i] if s1<=i1 else dims[i]-(s1-i1)
    
    subim[sst[0]:sen[0],sst[1]:sen[1],sst[2]:sen[2]]=voxels[ist[0]:ien[0],ist[1]:ien[1],ist[2]:ien[2]]
    img=nib.nifti1.Nifti1Image(subim,np.eye(4))
    #print ("image shape", ish, "subrange", bounds, "sst", sst, "sen", sen,"ist",ist,"ien",ien)
    return img
        

In [231]:
d='/data/TMORPH/I3CR/MW101_MR_tx1_GK/'
fim='gk2'
fmasks=['gk2_struct_Tum1_LFront','gk2_struct_Tum2_Vermis','gk2_struct_Tum3_RFront']
im,masks=split_image(d+fim+'.nii',[ d+f+'.nii' for f in fmasks ])




In [223]:
im.to_filename(d+'gk2_test.nii')
masks[0].to_filename(d+'gk2_mask_test.nii')

In [232]:
for i in range(len(fmasks)):
    subim,submask=get_subimages(im,masks[i])
    subim.to_filename(d+fim+'_'+fmasks[i])
    submask.to_filename(d+fmasks[i]+'_sub.nii')
    
    

output region: [[48, 118], [85, 155], [67, 137]]
output region: [[117, 137], [86, 106], [39, 59]]
output region: [[131, 151], [155, 175], [53, 73]]
